In [1]:
# Configure the necessary Python module imports
import dash_leaflet as dl
from jupyter_dash import JupyterDash
# import dash
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

# Other imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from crud_module import CRUD
import plotly.graph_objects

# MongoDB connection parameters
username = "aacuser"
password = "python132"
host = "nv-desktop-services.apporto.com"  # MongoDB server details
port = 31866
db_name = "AAC"  # database name
collection_name = "animals"  # collection name

# Create an instance of your CRUD class
crud_instance = CRUD(username, password, host, port, db_name, collection_name)

# Retrieve data from MongoDB
df = pd.DataFrame.from_records(crud_instance.query_documents({}))
df.drop(columns=["_id"], inplace=True)

# Define the Dash app layout
app = JupyterDash("Animal Shelter", suppress_callback_exceptions=True)
# app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div(id="hidden-div", style={"display": "none"}),
    
    #Add logo
    html.Div([
        html.Img(src="Grazioso Salvare Logo.png", alt="Logo", style={"height": "100px"}),
    ], style={"display": "flex", "justify-content": "center", "align-items": "center", "margin-top": "10px"}),
    
    #Add title
    html.Center(html.B(html.H1("Matthieu's Shelter Dashboard"))),
    html.Hr(),
    #Dropdown list
    dcc.Dropdown(
        id="filter-dropdown",
        options=[
            {"label": "Water Rescue", "value": "Water Rescue"},
            {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
            {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster or Individual Tracking"},
            {"label": "Reset", "value": "Reset"}
        ],
        value="Reset",
        multi=False
    ),
    html.Br(),
    #Data Table
    dash_table.DataTable(
        id="datatable-id",
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict("records"),
        row_selectable="single",  # Enable single-row selection
        # additional parameters
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
        ),

    html.Br(),
    html.Hr(),
    html.Div([
    # Pie chart
    dcc.Graph(id="pie-chart-figure", style={"width": "50%", "display": "inline-block"}),

    # Map
    html.Div(id="map-id", className="col s12 m6"),
    ], style={"display": "flex", "justify-content": "space-between"}),
])

def generate_map(dff, row):
    if len(dff) > 0 and 0 <= row < len(dff):
        return [
            dl.Map(style={"width": "1000px", "height": "500px"},
                   center=[30.75, -97.48], zoom=10, children=[
                       dl.TileLayer(id="base-layer-id"),
                       dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                                 children=[
                                     dl.Tooltip(dff.iloc[row, 4]),
                                     dl.Popup([
                                         html.H1("Animal Name"),
                                         html.P(dff.iloc[row, 9])
                                     ])
                                 ])
                   ])
        ]
    else:
        return []

@app.callback(
    [Output("datatable-id", "style_data_conditional"),
     Output("datatable-id", "data"),
     Output("datatable-id", "selected_rows"),
     Output("map-id", "children"),
     Output("pie-chart-figure", "figure")],
    [Input("filter-dropdown", "value"),
     Input("datatable-id", "derived_virtual_data"),
     Input("datatable-id", "derived_virtual_selected_rows")]
)
def update_data_map_and_chart(selected_filter, viewData, selected_rows):
    map_children = []  # Initialize map_children
    dff = pd.DataFrame()  # Initialize dff

    if viewData is not None:
        dff = pd.DataFrame(viewData)

    # Calculate breed percentages for the pie chart
    breed_counts = dff["breed"].value_counts()  # Use the correct column name
    breed_labels = breed_counts.index.tolist()
    breed_values = breed_counts.values.tolist()
    total_breeds = sum(breed_values)
    breed_percentages = [(count / total_breeds) * 100 for count in breed_values]

    pie_chart_figure = {
        "data": [
            {
                "labels": breed_labels,
                "values": breed_percentages,
                "type": "pie",
                "textinfo": "percent+label",
                "hole": 0.4,
            }
        ],
        "layout": {
            "title": "Breed Distribution",
            "margin": {"l": 30, "r": 30, "b": 30, "t": 50},
        },
    }

    if selected_filter == "Reset":
        filtered_data = df.to_dict("records")
        if selected_rows:
            row = selected_rows[0]
            map_children = generate_map(dff, row)
    else:
        if viewData is not None:
            dff = pd.DataFrame(viewData)

        # Apply filter criteria based on selected_filter
        if selected_filter == "Water Rescue":
            filtered_data = [record for record in dff.to_dict("records") if record["breed"] in ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"] and 26 <= record["age_upon_outcome_in_weeks"] <= 156 and record["sex_upon_outcome"] == "Intact Female"]
        elif selected_filter == "Mountain or Wilderness Rescue":
            filtered_data = [record for record in dff.to_dict("records") if record["breed"] in ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"] and 26 <= record["age_upon_outcome_in_weeks"] <= 156 and record["sex_upon_outcome"] == "Intact Male"]
        elif selected_filter == "Disaster or Individual Tracking":
            filtered_data = [record for record in dff.to_dict("records") if record["breed"] in ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"] and 20 <= record["age_upon_outcome_in_weeks"] <= 300 and record["sex_upon_outcome"] == "Intact Male"]
        else:
            filtered_data = df.to_dict("records")

        if selected_rows and len(dff) > 0:
            row = selected_rows[0]
            map_children = generate_map(dff, row)

    styles = []
    if viewData is not None:
        selected_columns = viewData[0].keys()
        styles = [{
            "if": {"column_id": i},
            "background_color": "#D2F3FF"
        } for i in selected_columns]

    return styles, filtered_data, selected_rows, map_children, pie_chart_figure

app.run_server()

/home/perrymatthieu_snhu/.local/lib/python3.9/site-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


Dash app running on http://127.0.0.1:27304/
